In [ ]:
using HTTP, MLJ, DataFrames, UrlDownload

In [ ]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data"
header = ["Class", "Alcool", "Malic acid", "Ash", "Alcalinity of ash",
          "Magnesium", "Total phenols", "Flavanoids",
          "Nonflavanoid phenols", "Proanthcyanins", "Color intensity",
          "Hue", "OD280/OD315 of diluted wines", "Proline"]
data = urldownload(url, true, format=:CSV, header=header);

In [ ]:
data = DataFrame(data)

# Manipulating data

In [ ]:
describe(data)

In [ ]:
methods(describe)

In [ ]:
using Plots
pyplot()
histogram(data[:,:Magnesium],leg=false)

In [ ]:
islabel = c -> c==:Class
isdata = c -> c!=:Class
y,X = unpack(data,islabel,isdata);

In [ ]:
y

In [ ]:
scitype(y)

In [ ]:
y = coerce(y,OrderedFactor);

In [ ]:
scitype(y)

In [ ]:
schema(X)

In [ ]:
coerce!(X,:Proline=>Continuous, :Magnesium=>Continuous);
schema(X)

# PCA

In [ ]:
models()

In [ ]:
models(matching(X))

In [ ]:
info("PCA")

In [ ]:
@load PCA pkg="MultivariateStats"

In [ ]:
pca_pipe = @pipeline Standardizer() MLJMultivariateStatsInterface.PCA(maxoutdim=3)

In [ ]:
pca = machine(pca_pipe,X)

In [ ]:
fit!(pca)

In [ ]:
report(pca)

In [ ]:
transform(pca)

In [ ]:
x1,x2,x3 = MLJ.transform(pca) |> eachcol;

In [ ]:
x1

In [ ]:
scatter(x1,x2,legend=false)

In [ ]:
colr = ["red" "blue" "magenta"]
scatter(x1,x2,color=colr[coerce(y,Count)],leg=false)

# k-Means clustering

In [ ]:
@load KMeans pkg="Clustering"

In [ ]:
km_pipe = @pipeline Standardizer() MLJClusteringInterface.KMeans()

In [ ]:
km = machine(km_pipe,X)

In [ ]:
fit!(km)

In [ ]:
report(km)

In [ ]:
ans.k_means

In [ ]:
clus = ans.assignments;

In [ ]:
colr = ["red" "blue" "magenta"]
scatter(x1,x2,color=colr[clus],leg=false)

In [ ]:
colr = ["red" "blue" "magenta"]
scatter(x1,x2,color=colr[coerce(y,Count)],leg=false)

# k-Nearest Neighbors

In [ ]:
@load KNNClassifier

In [ ]:
pipe = @pipeline Standardizer() NearestNeighborModels.KNNClassifier()

In [ ]:
train,test = partition(eachindex(y),0.8,shuffle=true)

In [ ]:
Xtrain,ytrain = X[train,:],y[train];
Xtest,ytest = X[test,:],y[test];

In [ ]:
knn = machine( pipe, Xtrain, ytrain )

In [ ]:
fit!(knn)

In [ ]:
evaluate!(knn,resampling=CV(nfolds=5))

In [ ]:
report(knn)

In [ ]:
predict(knn,Xtest)

In [ ]:
ŷ = predict_mode(knn,Xtest)

In [ ]:
count(ŷ.==ytest) / length(ytest)

In [ ]:
ConfusionMatrix()(predict_mode(knn,X),y)